In [ ]:
import os
import pandas as pd
from glob import glob

import matplotlib.pyplot as plt

In [ ]:
csv_files = glob("./storage/acciones/*.csv")

In [ ]:
matched_file_names = {os.path.basename(file).replace(".csv",""): file for file in csv_files}
matched_file_names

## Mapeo de valores para agregado a los valores de las acciones

In [ ]:
symbols_map = {
    "D": "Dominion Energy Inc.",
    "EXC": "Exelon Corp.",
    "NEE": "NextEra Energy Inc.",
    "SO": "Southern Co.",
    "DUK": "Duke Energy Corp."
}

In [ ]:
final_dataframe = []

for symbol_name, path_csv in matched_file_names.items():
    df = pd.read_csv(path_csv, sep=",")
    df["name"] = symbols_map[symbol_name]
    final_dataframe.append(df)

final_dataframe = pd.concat(final_dataframe).reset_index()

In [ ]:
final_dataframe.sample(10)

In [ ]:
# otra forma que yo recomiendo es entendiendo lo que hace los list comprehensions
# dfs = [
#     pd.read_csv(path_csv, sep=",").assign(name=symbols_map[symbol_name])
#     for symbol_name, path_csv in matched_file_names.items()
# ]

# final_dataframe = pd.concat(dfs, ignore_index=True)  

In [ ]:
# final_dataframe.sample(10)

## Solución Completa: Cálculo de Volatilidad y Retorno + Gráficos

- Calcular la volatilidad relativa ((High - Low) / Open)
- Calcular el retorno ((Close / Open) - 1)
- Graficar ambas métricas para comparar acciones.
- Identificar la mejor inversión (balance entre alto retorno y baja volatilidad).

In [ ]:
# Calculamos volatilidad y retorno
# volatilidad relativa ->  indicador de volatilidad que mide la variación de los precios de un activo financiero en un periodo de tiempo, usando la desviación estándar de los precios
final_dataframe["volatilidad_relativa"] = (final_dataframe["High"] - final_dataframe["Low"]) / final_dataframe["Open"]

# Retorno -> 
final_dataframe["retorno"] = (final_dataframe["Close"] / final_dataframe["Open"]) - 1

# Agrupamos por acción (name) y calculamos el promedio
metricas_por_accion = final_dataframe.groupby("name").agg({
    "volatilidad_relativa": "mean",
    "retorno": "mean"
}).reset_index()

print(metricas_por_accion)

## Graficos comparativos
### Volatilidad vs Retorno

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crear el scatterplot
plt.figure(figsize=(10, 6))
scatter = sns.scatterplot(
    data=metricas_por_accion,
    x="volatilidad_relativa",
    y="retorno",
    s=100,
    alpha=0.7,
    color="teal"
)

# Etiquetas y título
plt.xlabel("Volatilidad Relativa (Promedio)")
plt.ylabel("Retorno Promedio")
plt.title("Relación Volatilidad vs. Retorno por Acción")

# Agregar etiquetas con nombres
for i, row in metricas_por_accion.iterrows():
    plt.annotate(
        row["name"],
        (row["volatilidad_relativa"], row["retorno"]),
        textcoords="offset points",
        xytext=(5, 5),
        ha='left',
        fontsize=9
    )

plt.tight_layout()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ordenar los datos por nombre
df_ordenado = metricas_por_accion.sort_values("name", ascending=False)

# Crear figura con 2 subplots (uno encima del otro)
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 8), sharex=True)

# Volatilidad relativa
sns.barplot(
    data=df_ordenado,
    x="name", y="volatilidad_relativa",
    color="orange",
    ax=axes[0]
)
axes[0].set_title("Volatilidad Relativa por Acción")
axes[0].set_ylabel("Volatilidad")
axes[0].tick_params(axis='x', rotation=45)

# Retorno promedio
sns.barplot(
    data=df_ordenado,
    x="name", y="retorno",
    color="green",
    ax=axes[1]
)
axes[1].set_title("Retorno Promedio por Acción")
axes[1].set_ylabel("Retorno")
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


### **Volatilidad Relativa vs. Retorno: Explicación Simple**
#### _Un poquito de data contextual_
#### **1. Volatilidad Relativa**  
- **Definición**: Mide la variación porcentual entre el precio más alto (`High`) y más bajo (`Low`) de una acción en un día (o período), **normalizada por el precio de apertura** (`Open`).  

- **Interpretación**:  
  - **Valor alto** (ej: `0.05` o 5%): La acción tuvo mucha oscilación intraday (riesgo alto).  
  - **Valor bajo** (ej: `0.01` o 1%): La acción se mantuvo estable (riesgo bajo).  
- **Ejemplo**:  
  - Si `Open = 100`, `High = 105`, `Low = 102`:  

#### **2. Retorno**  
- **Definición**: Ganancia o pérdida porcentual de una acción desde la apertura (`Open`) hasta el cierre (`Close`).  
- **Interpretación**:  
  - **Positivo**: La acción subió (ej: `0.02` = +2%).  
  - **Negativo**: La acción bajó (ej: `-0.01` = -1%).  
- **Ejemplo**:  
  - Si `Open = 100` y `Close = 108`:  
---

### **¿Por qué son importantes?**  
- **Volatilidad Relativa**:  
  - Indica **riesgo**: Una acción con alta volatilidad puede generar grandes ganancias o pérdidas.  
  - Útil para traders que operan en corto plazo (ej: compran en `Low` y venden en `High`).  

- **Retorno**:  
  - Mide **rentabilidad**: Cuánto ganó un inversor que compró en `Open` y vendió en `Close`.  
  - Clave para inversores a largo plazo.  

---

### **Ejemplo Gráfico**  
Imagina estos datos para 3 acciones:  

| Acción | Volatilidad Relativa | Retorno |
|--------|----------------------|---------|
| **A**  | 4%                   | +5%     |
| **B**  | 1%                   | +2%     |
| **C**  | 8%                   | -3%     |

- **Mejor inversión**: Depende de tu perfil:  
  - **Conservador**: Elegirías **B** (baja volatilidad y retorno positivo).  
  - **Arriesgado**: Podrías considerar **A** (mayor retorno, pero más riesgo).  
  - **Evitarías**: **C** (pérdidas y alta volatilidad).  

---

### **Relación entre Ambos**  
- Generalmente, a mayor volatilidad, **mayor posibilidad de retornos altos** (pero también de pérdidas).  
- Una buena inversión suele buscar **alto retorno con baja volatilidad** (ej: acciones estables pero con crecimiento constante).  



In [ ]:
metricas_por_accion["ratio_retorno_volatilidad"] = metricas_por_accion["retorno"] / metricas_por_accion["volatilidad_relativa"]
mejor_inversion = metricas_por_accion.sort_values("ratio_retorno_volatilidad", ascending=False).iloc[0]

print(f"\nMejor inversión: {mejor_inversion['name']}")
print(f"Retorno: {mejor_inversion['retorno']:.2%}")
print(f"Volatilidad: {mejor_inversion['volatilidad_relativa']:.2%}")